# Execute SQL
[Getting Started with Python Integration to SAS® Viya® - Part 16 - Execute SQL](https://blogs.sas.com/content/sgf/2023/04/03/getting-started-with-python-integration-to-sas-viya-part-16-execute-sql/)

In [1]:
## Packages
import swat
import os
import pandas as pd
import numpy as np


## custom personal module to connect to my CAS environment
try:
    from casConnect import connect_to_cas 
except:
    print('CasConnect package not available')

## Make a Connection to CAS (REQUIRED: MODIFY CONNECTION INFORMATION)

##### To connect to the CAS server you will need:
1. the host name, 
2. the portnumber, 
3. your user name, and your password.

Visit the documentation [Getting Started with SAS® Viya® for Python](https://go.documentation.sas.com/doc/en/pgmsascdc/default/caspg3/titlepage.htm) for more information about connecting to CAS.

**Be aware that connecting to the CAS server can be implemented in various ways, so you might need to see your system administrator about how to make a connection. Please follow company policy regarding authentication.**

In [2]:
##
## Connect to CAS
##

## General connection syntax
# conn = swat.CAS(host, port, username, password)

## SAS Viya for Learners 3.5 connection
# hostValue = os.environ.get('CASHOST')
# portValue = os.environ.get('CASPORT')
# passwordToken=os.environ.get('SAS_VIYA_TOKEN')
# conn = swat.CAS(hostname=hostValue, port=portValue, password=passwordToken)

## Personal connection
try:
    conn = connect_to_cas()
    print('CAS connection succesful')
    print(conn)
except:
    print('No connection')
    pass

CAS connection succesful
CAS('ssemonthly.demo.sas.com', 443, protocol='https', name='py-session-1', session='e6591086-4529-0f45-832b-c57baf1db151')


## Create demo CAS tables

In [3]:
def createDemoData(connectionObject):
    '''Specify your CAS connectin object as the parameter'''
    
    ## Load the rand_retaildemo.sashdat file from the samples caslib
    connectionObject.loadTable(path = 'RAND_RETAILDEMO.sashdat', caslib = 'samples',
                               casout = {
                                   'caslib':'casuser',
                                   'replace':True
                               })
    
    ## Create a sample DataFrame, then load it to CAS
    myData = {
        'loyalty_id':[0,1],
        'loyalty_value':['No','Yes'],
    }
    df = pd.DataFrame(myData)
    connectionObject.upload_frame(df, 
                                  casout = {
                                        'name':'loyalty_lookup_tbl',
                                        'caslib':'casuser',
                                        'replace':True
                                   })
    
    ## View CAS tables
    tableInfo = connectionObject.tableInfo(caslib = 'casuser')
    return tableInfo

createDemoData(conn)   

NOTE: Cloud Analytic Services made the file RAND_RETAILDEMO.sashdat available as table RAND_RETAILDEMO in caslib CASUSER(Peter.Styliadis@sas.com).
NOTE: Cloud Analytic Services made the uploaded file available as table LOYALTY_LOOKUP_TBL in caslib CASUSER(Peter.Styliadis@sas.com).
NOTE: The table LOYALTY_LOOKUP_TBL has been created in caslib CASUSER(Peter.Styliadis@sas.com) from binary data uploaded to Cloud Analytic Services.


,Name,Rows,Columns,IndexedColumns,Encoding,CreateTimeFormatted,ModTimeFormatted,AccessTimeFormatted,JavaCharSet,CreateTime,Repeated,View,MultiPart,SourceName,SourceCaslib,Compressed,Creator,Modifier,SourceModTimeFormatted,SourceModTime
0,RAND_RETAILDEMO,930046,40,0,utf-8,2023-05-09T18:14:27+00:00,2023-05-09T18:14:27+00:00,2023-05-09T18:14:27+00:00,UTF8,1.999275e+09,0,0,0,RAND_RETAILDEMO.sashdat,Samples,1,Peter.Styliadis@sas.com,,2021-12-18T18:35:04+00:00,1.955472e+09
1,LOYALTY_LOOKUP_TBL,2,2,0,utf-8,2023-05-09T18:14:28+00:00,2023-05-09T18:14:28+00:00,2023-05-09T18:14:28+00:00,UTF8,1.999275e+09,0,0,0,,,0,Peter.Styliadis@sas.com,,2023-05-09T18:14:28+00:00,1.999275e+09


## Load the FedSQL CAS action set

[FedSQL CAS action set documentation](https://go.documentation.sas.com/doc/en/pgmsascdc/default/caspg/cas-fedsql-execdirect.htm#PYTHON.cas-fedsql-execdirect-method)

In [4]:
conn.loadActionSet('fedSQL')

NOTE: Added action set 'fedSQL'.


[actionset]

 'fedSQL'

+ Elapsed: 0.0118s, user: 0.00221s, sys: 0.029s, mem: 1.2mb

In [5]:
conn.help(actionset='fedSQL')

NOTE: Information for action set 'fedSQL':
NOTE:    fedSql
NOTE:       execDirect - Submits a SAS FedSQL language statement for immediate execution


,name,description
0,execDirect,Submits a SAS FedSQL language statement for im...


## Perform a simple query

In [6]:
conn.tableInfo(caslib = 'casuser')

,Name,Rows,Columns,IndexedColumns,Encoding,CreateTimeFormatted,ModTimeFormatted,AccessTimeFormatted,JavaCharSet,CreateTime,Repeated,View,MultiPart,SourceName,SourceCaslib,Compressed,Creator,Modifier,SourceModTimeFormatted,SourceModTime
0,RAND_RETAILDEMO,930046,40,0,utf-8,2023-05-09T18:14:27+00:00,2023-05-09T18:14:27+00:00,2023-05-09T18:14:27+00:00,UTF8,1.999275e+09,0,0,0,RAND_RETAILDEMO.sashdat,Samples,1,Peter.Styliadis@sas.com,,2021-12-18T18:35:04+00:00,1.955472e+09
1,LOYALTY_LOOKUP_TBL,2,2,0,utf-8,2023-05-09T18:14:28+00:00,2023-05-09T18:14:28+00:00,2023-05-09T18:14:28+00:00,UTF8,1.999275e+09,0,0,0,,,0,Peter.Styliadis@sas.com,,2023-05-09T18:14:28+00:00,1.999275e+09


### Preview the RAND_RETAILDEMO CAS table

In [7]:
previewRetail = '''
    select *
        from casuser.RAND_RETAILDEMO
        limit 5;
'''
conn.execDirect(query = previewRetail)

,CustID,bucket,age,loyalty_card,Department,brand_name,Storechain,ChannelType,Class,StoreNum,Region_Long,Region_2,Region_2_Lat,Region_2_Long,State,State_Lat,State_Long,age_bucket,Storechain1,brand_name1
0,1083863.0,1.0,NaN,0.0,health,Pine,GRAND,Resale,oral care,736.0,-74.50,US Atlantic Coast,40.3,-74.50,NY,41.5435,-74.6941,18-25,FAST,Oak
1,1083863.0,1.0,NaN,0.0,kids,Pine,GRAND,Resale,kids_swimwear,736.0,-74.50,US Atlantic Coast,40.3,-74.50,NY,41.5435,-74.6941,36-45,GRAND,Pine
2,1083863.0,1.0,NaN,0.0,men,Pine,GRAND,Resale,men_slippers,736.0,-84.75,US Midwest,43.3,-84.75,IL,41.2340,-88.3961,56-65,MODA,Maple
3,1083863.0,2.0,NaN,0.0,men,Pine,GRAND,Internet,men_underwear,724.0,-74.50,US Atlantic Coast,40.3,-74.50,NJ,40.4403,-74.4296,46-55,MODA,Maple
4,1083883.0,2.0,23.0,1.0,electronics,Pine,GRAND,Store,DVD & Blu-ray,757.0,-84.75,US Midwest,43.3,-84.75,KS,38.4575,-96.5830,26-35,GRAND,Pine


### Preview the LOYALTY_LOOKUP_TBL CAS table

In [8]:
previewLoyalty = '''
    select *
        from casuser.LOYALTY_LOOKUP_TBL
        limit 5;
'''
conn.execDirect(query = previewLoyalty)

,loyalty_id,loyalty_value
0,0.0,No
1,1.0,Yes


## Perform a join

Perform an inner join:
- the **RAND_RETAILDEMO** CAS table from the **Casuser** caslib using the **loyalty_card** column
- the **LOYALTY_LOOKUP_TBL** CAS table from the **Casuser** caslib using the **loyalty_id** column
- keep the **CustID**, **age**, **Department**, **loyalty_card** and **loyalty_value** columns
- limit the join to 5 rows to confirm it works as expected

In [9]:
joinData = '''
    select retail.CustID,
           retail.age,
           retail.Department,
           retail.loyalty_card,
           loyalty.loyalty_value
        from casuser.RAND_RETAILDEMO as retail inner join
             casuser.LOYALTY_LOOKUP_TBL as loyalty
        on retail.loyalty_card = loyalty.loyalty_id
        limit 5;
'''

conn.execDirect(query = joinData)

,CustID,age,Department,loyalty_card,loyalty_value
0,29555833.0,NaN,women,0.0,No
1,29555911.0,24.0,grocery,1.0,Yes
2,29555944.0,NaN,women,0.0,No
3,29555981.0,NaN,women,0.0,No
4,29556043.0,18.0,women,1.0,Yes


## Create a new CAS table using the join

In [10]:
createTable = '''
    create table loyalty_final{options replace=True} as
    select retail.CustID,
           retail.age,
           retail.Department,
           retail.loyalty_card,
           loyalty.loyalty_value
        from casuser.RAND_RETAILDEMO as retail inner join
             casuser.LOYALTY_LOOKUP_TBL as loyalty
        on retail.loyalty_card = loyalty.loyalty_id;
'''

conn.execDirect(query = createTable)

NOTE: Table LOYALTY_FINAL was created in caslib CASUSER(Peter.Styliadis@sas.com) with 930046 rows returned.


+ Elapsed: 0.341s, user: 2.6s, sys: 1.86s, mem: 1.15e+03mb

In [11]:
conn.tableInfo(caslib = 'casuser')

,Name,Rows,Columns,IndexedColumns,Encoding,CreateTimeFormatted,ModTimeFormatted,AccessTimeFormatted,JavaCharSet,CreateTime,Repeated,View,MultiPart,SourceName,SourceCaslib,Compressed,Creator,Modifier,SourceModTimeFormatted,SourceModTime
0,RAND_RETAILDEMO,930046,40,0,utf-8,2023-05-09T18:14:27+00:00,2023-05-09T18:14:27+00:00,2023-05-09T18:14:34+00:00,UTF8,1.999275e+09,0,0,0,RAND_RETAILDEMO.sashdat,Samples,1,Peter.Styliadis@sas.com,,2021-12-18T18:35:04+00:00,1.955472e+09
1,LOYALTY_LOOKUP_TBL,2,2,0,utf-8,2023-05-09T18:14:28+00:00,2023-05-09T18:14:28+00:00,2023-05-09T18:14:34+00:00,UTF8,1.999275e+09,0,0,0,,,0,Peter.Styliadis@sas.com,,2023-05-09T18:14:28+00:00,1.999275e+09
2,LOYALTY_FINAL,930046,5,0,utf-8,2023-05-09T18:14:34+00:00,2023-05-09T18:14:34+00:00,2023-05-09T18:14:34+00:00,UTF8,1.999275e+09,0,0,0,,,0,Peter.Styliadis@sas.com,,,NaN


In [12]:
loyaltyInfo = conn.CASTable('LOYALTY_FINAL',caslib = 'casuser')
loyaltyInfo.head()

,CustID,age,Department,loyalty_card,loyalty_value
0,24731435.0,38.0,grocery,1.0,Yes
1,24731435.0,38.0,health,1.0,Yes
2,24731435.0,38.0,health,1.0,Yes
3,24731435.0,38.0,men,1.0,Yes
4,24731437.0,NaN,kids,0.0,No


## Terminate the CAS session

In [13]:
conn.terminate()